<a href="https://colab.research.google.com/github/kk412027247/nlp/blob/main/test_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
!wget https://www.wangluoguimi.com/news/news-headlines.tsv

In [ ]:
chars = sorted(set("abcdefghijklmnopqrstuvwxyz0123456789 -,;.!?:’’’/\|_@#$%ˆ&*˜‘+-=()[]{}' ABCDEFGHIJKLMNOPQRSTUVWXYZ"))
chars = list(chars)
EOS = '<EOS>'
UNK = "<UNK>"
PAD = "<PAD>"
chars.append(UNK)
chars.append(EOS)


chars.insert(0, PAD)

char2idx = {u:i for i, u in enumerate(chars)}
idx2char = np.array(chars)

def char_idx(c):
    if c in chars:
        return char2idx[c]
    return char2idx[UNK]

In [ ]:
vocab_size = len(chars)
embedding_dim = 256
rnn_units = 1024
BATCH_SIZE = 1

def build_gen_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model
gen_model = build_gen_model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE)

In [ ]:
checkpoint_dir = './training_checkpoints/2020-Oct-01-14-29-55'
gen_model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
gen_model.build(tf.TensorShape([1, None]))

In [ ]:
def generate_text(model, start_string, temperature=0.7, num_generate=75):
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature 
    predicted_id = tf.random.categorical(predictions, hum_samples=1)[-1, 0].numpy()
    input_eval = tf.expand_dims([predicted_id], 0)
    text_generated.append(idx2char[predicted_id])
  return (start_string + ''.join(text_generated))